In [65]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import re


bins = 25
total_balls = 25*25

delta_array = [2,3,4,5,6,7,8,9,10]
beta_array = [0.2, 0.4, 0.6, 0.8]



In [67]:
#for balls in range(0,total_balls+1,bins):
for beta in beta_array:
    subprocess.run(["./galtonBoard", "-B", str(beta), "-K 2"])

Parsed options:
Allocation scheme: B, Value: 0.2
Experiment type: K, Value: 2
Parsed options:
Allocation scheme: B, Value: 0.4
Experiment type: K, Value: 2
Parsed options:
Allocation scheme: B, Value: 0.6
Experiment type: K, Value: 2
Parsed options:
Allocation scheme: B, Value: 0.8
Experiment type: K, Value: 2


In [63]:
folder_path = 'comparisons-k-delta/'

data = {}

pattern = re.compile(r"D(\d+)_K(\d+)\.csv")
#pattern = re.compile(r"D(\d+)_00.csv")

for filename in os.listdir(folder_path):
    match = pattern.match(filename)
    if match:
        delta_value = int(match.group(2))
        batch_size = int(match.group(1))
        
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        # Extract values for n = 25 and N = 625
        value_n_25 = df[df['balls'] == 25].iloc[0]['averageMaxLoad']
        value_N_625 = df[df['balls'] == 625].iloc[0]['averageMaxLoad']
        
        if batch_size not in data:
            data[batch_size] = {}
        
        data[batch_size][delta_value] = (value_n_25, value_N_625)
        #data[beta_value] = (value_n_25, value_N_625)


latex_output = ""
for batch_size in sorted(data.keys()):
    row = f"\\textbf{{{batch_size}}}"
    for delta_value in sorted(data[batch_size].keys()):
        n_25, N_625 = data[batch_size][delta_value]
        row += f" & {n_25} & {N_625}"
    latex_output += row + " \\\\ \\hline\n"


print(latex_output)

\textbf{2} & 2.21 & 4.44 & 1.83 & 2.36 \\ \hline
\textbf{3} & 2.0 & 3.19 & 1.47 & 1.67 \\ \hline
\textbf{4} & 1.91 & 2.95 & 1.35 & 1.41 \\ \hline
\textbf{5} & 1.75 & 2.66 & 1.25 & 1.36 \\ \hline
\textbf{6} & 1.76 & 2.51 & 1.2 & 1.29 \\ \hline
\textbf{7} & 1.82 & 2.41 & 1.18 & 1.27 \\ \hline
\textbf{8} & 1.79 & 2.61 & 1.13 & 1.11 \\ \hline
\textbf{9} & 1.85 & 2.27 & 1.13 & 1.08 \\ \hline
\textbf{10} & 1.9 & 2.32 & 0.97 & 1.13 \\ \hline



In [39]:
import csv

for beta_value in {beta for batch in data.values() for beta in batch.keys()}:
    filename = f'beta_{beta_value}_N625_values.csv'
    
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['batch', 'value']) 

        for batch_size, beta_dict in sorted(data.items()):
            if beta_value in beta_dict:
                _, value_N_625 = beta_dict[beta_value]
                writer.writerow([batch_size, value_N_625])

    print(f"New CSV created for beta {beta_value}: {filename}")